In [ ]:
from selenium import webdriver
import subprocess
import chromedriver_autoinstaller
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import chromedriver_autoinstaller
    
from selenium.webdriver.common.keys import Keys

import time
import datetime as dt
import schedule

import pandas as pd
import numpy as np
from pprint import pprint
import urllib.request
import json
import glob
import sys
import os

import warnings
warnings.filterwarnings(action='ignore')

pd.set_option('display.max_columns', 250)
pd.set_option('display.max_rows', 250)
pd.set_option('display.width', 100)

pd.options.display.float_format = '{:.2f}'.format

class NaverDataLabOpenAPI():
    """
    네이버 데이터랩 오픈 API 컨트롤러 클래스
    """

    def __init__(self, client_id, client_secret):
        """
        인증키 설정 및 검색어 그룹 초기화
        """
        self.client_id = client_id
        self.client_secret = client_secret
        self.keywordGroups = []
        self.url = "https://openapi.naver.com/v1/datalab/search"

    def add_keyword_groups(self, group_dict):
        """
        검색어 그룹 추가
        """

        keyword_gorup = {
            'groupName': group_dict['groupName'],
            'keywords': group_dict['keywords']
        }
        
        self.keywordGroups.append(keyword_gorup)
        print(f">>> Num of keywordGroups: {len(self.keywordGroups)}")
        
    def get_data(self, startDate, endDate, timeUnit, device, ages, gender):
        """
        요청 결과 반환
        timeUnit - 'date', 'week', 'month'
        device - None, 'pc', 'mo'
        ages = [], ['1' ~ '11']
        gender = None, 'm', 'f'
        """

        # Request body
        body = json.dumps({
            "startDate": startDate,
            "endDate": endDate,
            "timeUnit": timeUnit,
            "keywordGroups": self.keywordGroups,
            "device": device,
            "ages": ages,
            "gender": gender
        }, ensure_ascii=False)
        
        # Results
        request = urllib.request.Request(self.url)
        request.add_header("X-Naver-Client-Id",self.client_id)
        request.add_header("X-Naver-Client-Secret",self.client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            # Json Result
            result = json.loads(response.read())
            
            period = []
            ratio = []
            title = []
#             pprint(result)
            if result['results'][0]['data']:
                for val in result['results']:
                    period.append(val['data'][-1]['period'])
                    ratio.append(val['data'][-1]['ratio'])
                    title.append(val['title'])
            
            df = pd.DataFrame({"title" : title, 'period' : period, "ratio" : ratio})
            
        else:
            print("Error Code:" + rescode)
            
        return df

keyword_group_set = {
    'keyword_group_1': {'groupName': "애플", 'keywords': ["애플"]},
}
def run():
    print("시작시작시작")
    try:
        subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
    except:
        subprocess.Popen(r'C:\Program Files (x86)\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp1"')  # 디버거 크롬 구동
    option = Options()
    option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    # option.add_argument("--headless")

    chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]
    try:
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)
    except:
        chromedriver_autoinstaller.install('./')
        driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

    #####크롤링
    
    ##비건
    
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%EB%B9%84%EA%B1%B4&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_l=[]
    for i in range(20):
        title_l.append(title[i].text)
        print(title_l[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('비건')
    search.send_keys(Keys.ENTER)
    title2=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_b=[]

    for i in range(len(title2)):
        title_b.append(title2[i].text)
        print(title_b[i])
        
    ##채식
    driver.get('https://blackkiwi.net/service/keyword-analysis?keyword=%EC%B1%84%EC%8B%9D&platform=naver')
    time.sleep(10) #혹시 모를 에러 방지를 위한 적절한 wait 넣어주기
    title3=driver.find_elements(By.CSS_SELECTOR,'table a')
    title_3=[]
    for i in range(20):
        title_3.append(title3[i].text)
        print(title_3[i])
    driver.implicitly_wait(10)
    driver.get('http://surffing.net/MainSeoSearch.do')
    search=driver.find_element(By.ID,'saerchKeyword')
    search.send_keys('채식')
    search.send_keys(Keys.ENTER)
    title4=driver.find_elements(By.CSS_SELECTOR,'tr>td.center:nth-child(1)')
    title_4=[]

    for i in range(len(title4)):
        title_4.append(title4[i].text)
        print(title_4[i])
    
    title_l.extend(title_b)
    title_l.extend(title_3)
    title_l.extend(title_4)
    list_a=title_l
    
    ##날짜지정
    today = dt.date.today()
    monday=today - dt.timedelta(days=today.weekday())
    end=monday.strftime('%Y-%m-%d')
    print(end)
    th=dt.timedelta(days=30)
    start=monday-th
    start=start.strftime('%Y-%m-%d')
        
    #####크롤링end
    cnt=0
    while(True):
        if len(list_a)>0 :
            for idx, (key, val) in enumerate(keyword_group_set.items()):
                val['groupName'] = list_a[idx]
                val['keywords'] = [list_a[idx]]
            del list_a[:1]
            cnt+=1
        else:
            break
    #     pprint(keyword_group_set)

            # API 인증 정보 설정
        client_id = "xTsOEpqU869lYEaqyaU6"
        client_secret = "kkp_ZP2j9T"

        # 요청 파라미터 설정
        startDate = start
        endDate = end
        timeUnit = 'week'
        device = ''
        ages = []
        gender = ''

        # 데이터 프레임 정의
        naver = NaverDataLabOpenAPI(client_id=client_id, client_secret=client_secret)

        naver.add_keyword_groups(keyword_group_set['keyword_group_1'])

        df = naver.get_data(startDate, endDate, timeUnit, device, ages, gender)

        if(cnt>1):
            df1=pd.concat([df1,df])
        else:
            df1=df
#     print(df1)
    driver.close()
    #검색주간과 같은 데이터만 추출
    df1=df1[df1['period']==end]
    #ratio높은순으로 정렬
    df2=df1.sort_values('ratio',ascending=False)
    #ratio 삭제
    df2=df2.drop([df2.columns[2]],axis=1)
    #인덱싱해주기
    df2.reset_index(drop=True,inplace=True)
    #10위까지 저장
    data=df2.loc[:9]
    print(data)
    return data

# step3.실행 주기 설정
schedule.every().monday.at("02:00").do(run)
# schedule.every().day.at("12:53").do(run)
# step4.스캐쥴 시작
while True:
    schedule.run_pending()
    time.sleep(1)

시작시작시작
비건 젤리
비건 식품
비건 요리
비건스콘
비건 버터
비건 가죽
비건 김밥
비건 인증
비건 과자
비건 빵집
비건 치즈
비건 라면
비건 만두
비건 레더
비건쿠키
비건 가방
비건 음식
비건식당
비건빵
비건 뜻
비건
비건가방
비건박람회
콩고기
비건백
비건화장품
강남맛집
천연화장품
대체육
비건식당
전시회
비건베이커리
신사역맛집
비건음식
전시
비건가죽
채식 단계
채식 반찬
채식 단백질
야채 식단
생채식
현미채식
채식 라면
채식한끼
채식주의
채식 만두
채식 요리
채식부페
채식식당
채식뷔페
채식 식단
보식
생식
단식
선식
한식
채식
콩고기
사찰음식
비건
대체육
서울가볼만한곳
강남맛집
반찬메뉴
제철음식
서울근교당일치기여행
비욘드미트
비건도시락
비건식당
장담그기
한식
김장체험
2022-08-22
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGroups: 1
>>> Num of keywordGro